In [ ]:
import pandas as pd
import os
import subprocess

# A list of the channels you want to download data for, one channel name per line
channels = pd.read_csv('PATH/TO/channel_names.txt', delimiter='\n', header=None)

# These are official Twitch API keys you can get from https://dev.twitch.tv/console by registering an application
CLIENT_ID = 'PUT YOUR CLIENT ID HERE'
CLIENT_SECRET = 'PUT YOUR CLIENT SECRET HERE'

OUTPUT_PATH = 'PATH/TO/THIS/PROJECT/Downloads/My Scrapes/' # Ex: C:/User/Projects/EmoteRecommender/Downloads/My Scrapes/

In [ ]:
"""
This portion requires the following module from GitHub:
Twitch Chat Downloader - https://github.com/PetterKraabol/Twitch-Chat-Downloader
"""

# Download chat logs from the channel's saved VODs
num_vods = 10

for streamer in channels[0]:
    print('Downloading VOD chats from ' + streamer)

    # Call the tcd module using your system
    os.system('tcd --channel ' + streamer + \
        ' --first=' + str(num_vods) + \
        ' --output_PATH "' + OUTPUT_PATH + streamer + '"')

# Note
Be sure to run the downloaded chat logs through regex_to_csv.sh to convert the .txt files to .csv so it works with the rest of the files in this project.

The script uses 'FILES="./data/\*/\*.txt"' to locate files, so make sure your file structure is compatible or edit this line in the script.

In [ ]:
"""
This portion requires the following module from GitHub:
EmoteDownloader - https://github.com/Daniel2193/EmoteDownloader

Note that this repository requires that you build the program from the source code, as it does not come with a .dll or .exe
"""

# Download channel emotes
skip = True

for streamer in channels[0]:
    # if [Error handling condition]:
    #     skip = False
    # if skip:
    #     continue

    print('Downloading emotes from ' + streamer)
    folder = OUTPUT_PATH + streamer
    #os.makedirs(folder[1:], exist_ok=True)

    console_command = 'dotnet "PATH/TO/EmoteDownloader.dll" --client_id ' + CLIENT_ID + ' --client_secret ' + CLIENT_SECRET + \
        ' --channel_names ' + streamer + \
        ' --output_dir ' + folder + '/"'

    try:
        subprocess.run(console_command + ' -p twitch', shell=True, check=True)
        #print(console_command + ' -p twitch')
    except:
        print('Error:',streamer,'twitch')

    try:
        subprocess.run(console_command + ' -p bttv', shell=True, check=True)
        #print(console_command + ' -p bttv')
    except:
        print('Error:',streamer,'bttv')
    
    # !!! FFZ doesn't work with this module, do it manually in the next cell instead
    # try:
    #     subprocess.run(console_command + ' -p ffz', shell=True, check=True)
    #     #print(console_command + ' -p ffz')
    # except:
    #     print('Error:',streamer,'ffz')

In [ ]:
import requests
import time

ffz_emotes = {}

for channel_name in channels[0]:
    # Fetch the channel ID
    response = requests.get(f"https://api.frankerfacez.com/v1/room/{channel_name}")

    # Hit the rate limit
    while response.status_code == 429:
        # Retry after some time
        print("API rate limit exceeded. Retrying after 60 seconds...")
        time.sleep(60)
        response = requests.get(f"https://api.frankerfacez.com/v1/room/{channel_name}")

    if response.status_code != 200:
        print(f"Could not find ID for channel '{channel_name}'")
    else:
        # Parse response and fetch the emotes for the channel
        data = response.json()
        set_id = data.get("room", {}).get("set", {})
        emotes = data.get("sets", {}).get(f'{set_id}', {}).get("emoticons", [])

        # Save emotes per channel
        emote_list = set()
        for emote in emotes:
            # print(f"Emote code: {emote.get('name')}, ID: {emote.get('id')}")
            emote_list.add(emote.get('name'))

        ffz_emotes[channel_name] = emote_list

In [19]:
# Might as well build the FFZ emote registry while we have everything loaded in memory
import pickle

df = pd.DataFrame.from_dict(ffz_emotes, orient='index').T

with open('ffz_dict','wb') as f:
    pickle.dump(df,f)